In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
import numpy as np
import datetime as dt
from bs4 import BeautifulSoup as bs
import requests
import time
from sqlalchemy import *
from sqlalchemy import create_engine
import sqlite3 as sql
import operator
import re
import os
os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs')

In [2]:
def latest_sec_db_links(formtype, year, qtr, engine):
    
    if formtype == 'F4':
        table_name = 'F4Links'
    elif formtype == '10Q':
        table_name = 'Links10Q'
    elif formtype == '10K':
        table_name = 'Links10K'
    else:
        print('formtypes are F4 10Q and 10K')
        return
    
    query = 'SELECT * FROM {0} WHERE Year = {1} AND Quarter = {2}'.format(table_name, year, qtr)
    
    db_table = pd.read_sql_query(query, con=engine, index_col = 'idx')
    
    db_table['Date Filed'] = pd.to_datetime(db_table['Date Filed'])
    
    return db_table.reset_index(drop = True)


def fin_statement_search(text_blob):
    if 'statements of operations' in text_blob.lower():
        return True
    elif 'balance sheets' in text_blob.lower():
        return True
    elif 'statements of cash flows' in text_blob.lower():
        return True
    elif 'statements of stockholders' in text_blob.lower():
        return True
    else:
        return False


In [3]:
def retrieve_statements(sec_url):
    
    start_time = time.time()
    
    def check_statement(b):
        # Checking for Balance Sheet
        if 'balancesheets' in b or 'balance sheets' in b or 'statements of financial conditions' in b:
            return 'balancesheet'
        elif 'statements of operations' in b or 'statements ofoperations' in b or 'statements of income' in b or 'statements ofincome' in b:
            return 'incomestatement'
        elif 'statements of stockholders' in b or 'statements ofstockholders' in b:
            return 'stockholdersequity'
        elif 'statements of cash flows' in b or 'statements of cashflows' in b or 'statements ofcash flows' in b or 'statements ofcashflows' in b:
            return 'cashflowstatement'
        else:
            return ''
        
    sec_req = requests.get(sec_url).text.lower()
    all_hrs = [m.start() for m in re.finditer('<hr', sec_req)]
    
    sec_pages = [bs(sec_req[all_hrs[i - 1]:all_hrs[i]], 'lxml') for i in range(1,len(all_hrs))]
    #sec_pages = list(filter(lambda x: x.find('table') != None, sec_pages))
    
    i = 0

    for page in sec_pages:
        titles = [x.text.lower().strip().replace('\n','') for x in page.find_all('b')]
        for b in titles:
            if check_statement(b) == 'balancesheet':
                balance_sheet = page
                break
            elif check_statement(b) == 'incomestatement':
                income_statement = page
                break
            elif check_statement(b) == 'stockholdersequity':
                stockholders_equity = page
                break
            elif check_statement(b) == 'cashflowstatement':
                cashflow_statement = page
                break
    print('Completed in {} seconds'.format(time.time() - start_time))
    
    try:
        return balance_sheet, income_statement, cashflow_statement, stockholders_equity
    except:
        try:
            return balance_sheet, income_statement, cashflow_statement
        except:
            return None

def parse_statement_table(curr_statement, cik, filedate):
    start_time = time.time()
    for b in [x.text.strip().lower() for x in curr_statement.find_all('b')]:
        if 'hundreds' in b:
            multiplier = 10**2
            break
        elif 'thousands' in b:
            multiplier = 10**3
            break
        elif 'millions' in b:
            multiplier = 10**6
            break
        else:
            multiplier = 1

    table = curr_statement.find('table')

    rows = []

    multiple_periods_exist = False

    for row in table.find_all('tr'):
        columns = [re.sub(' +',' ',x.text.strip().lower().replace('$', '').replace('\n',' ')).replace('\x97','-').replace('\x92','') for x in row.find_all('td')]
        columns = list(filter(lambda x: x != '' and x != ')' and x != '(', columns))
        if len(columns) > 1:

            if list(filter(lambda x: 'three month' in x, columns)) != []:
                check_col = columns
                multiple_periods_exist = True
            elif multiple_periods_exist == False:
                check_col = []

            try:
                columns = [dt.datetime.strptime(x, '%B %d, %Y').date() for x in columns]
                rows.append(columns)
                dates = columns
            except:
                rows.append(columns)

    frame_width = max([len(x) for x in rows])
    curr_statement_df = pd.DataFrame(columns = range(frame_width-1))
    for i, row in enumerate(rows):

        if len(row) == frame_width:
            curr_statement_df.loc[row[0]] = [float(x.replace('-','0').replace('(','-').replace(',',''))*multiplier for x in row[1:]]

    if check_col != []:
        dates = dates[2*check_col.index("three months ended"):check_col.index("three months ended") + 2]

    df_column, value = max(enumerate(dates), key=operator.itemgetter(1))
    curr_statement_df = curr_statement_df[[df_column]]
    curr_statement_df.columns = ['values']
    curr_statement_df['endingQuarter'] = value
    curr_statement_df['CIK'] = cik
    curr_statement_df['DateFiled'] = filedate
    
    print('Completed in {} seconds'.format(time.time() - start_time))
    return curr_statement_df

In [ ]:
sec_engine = create_engine('sqlite:///SEC_txt.db', echo=False)
latest_db_10q = latest_sec_db_links('10Q', 2003, 3, sec_engine)

In [ ]:
sec_fs_engine = create_engine('sqlite:///SEC_statements.db', echo=False)

for idx in latest_db_10q.index:
    curr_filing = latest_db_10q.loc[idx,:]
    curr_filing['Date Filed'] = pd.to_datetime(curr_filing['Date Filed']).date()
    sec_url = 'https://www.sec.gov/Archives/' + curr_filing['Filename']
    curr_cik = curr_filing['CIK']
    file_date = curr_filing['Date Filed']

    failed_completely = False
    
    try:
        bs, ins, cf, se = retrieve_statements(sec_url)
    except:
        try:
            bs, ins, cf = retrieve_statements(sec_url)
        except:
            failed_completely = True
            pd.DataFrame(curr_filing).T.to_sql('FailedFilings', con=sec_fs_engine, if_exists='append', index_label = 'idx')

    if failed_completely == False:
        try:
            curr_balance_sheet = parse_statement_table(bs,curr_cik,file_date)
            curr_balance_sheet.to_sql('BalanceSheets', con=sec_fs_engine, if_exists='append', index_label = 'idx')
        except:
            pd.DataFrame(curr_filing).T.to_sql('FailedBalanceSheets', con=sec_fs_engine, if_exists='append', index_label = 'idx')

        try:
            curr_income_statement = parse_statement_table(ins,curr_cik,file_date)
            curr_income_statement.to_sql('IncomeStatements', con=sec_fs_engine, if_exists='append', index_label = 'idx')
        except:
            pd.DataFrame(curr_filing).T.to_sql('FailedIncomeStatements', con=sec_fs_engine, if_exists='append', index_label = 'idx')

        try:
            curr_cashflows = parse_statement_table(cf,curr_cik,file_date)
            curr_cashflows.to_sql('CashFlows', con=sec_fs_engine, if_exists='append', index_label = 'idx')
        except:
            pd.DataFrame(curr_filing).T.to_sql('FailedCashFlows', con=sec_fs_engine, if_exists='append', index_label = 'idx')

        try:
            curr_stockequity = parse_statement_table(se,curr_cik,file_date)
            curr_stockequity.to_sql('StockholdersEquity', con=sec_fs_engine, if_exists='append', index_label = 'idx')
        except:
            pd.DataFrame(curr_filing).T.to_sql('FailedStockholdersEquity', con=sec_fs_engine, if_exists='append', index_label = 'idx')

In [4]:
sec_url = 'https://www.sec.gov/Archives/edgar/data/225261/0001047469-03-023135.txt'

In [6]:
bs, ins, cf = retrieve_statements(sec_url)

Completed in 0.432234525680542 seconds


In [9]:
cf

<html><body><hr noshade=""/>
<!-- zeq.=1,seq=4,efw="2114096",cp="dmi furniture inc.",dn="1",chk=680605,folio='4',file='disk016:[03chi2.03chi3532]dg3532a.;4',user='dholbro',cd=';1-jul-2003;02:48' -->
<a name="page_dg3532_1_5"> </a>
<p align="center"><font size="2"><a name="dg3532_dmi_furniture,_inc._consolidat__dmi03431"> </a>
<a name="toc_dg3532_2"> </a>
<br/></font><font size="2"><b>dmi furniture, inc.<br/>  consolidated statements of cash flows<br/>  (amounts in thousands)<br/>  (unaudited)    <br/> </b></font></p>
<!-- user-specified tagged table -->
<table border="0" cellpadding="0" cellspacing="0" width="85%">
<tr valign="bottom">
<th align="left" colspan="4"><font size="2"> </font><br/></th>
<th width="2%"><font size="1"> </font></th>
<th align="center" colspan="5"><font size="1"><b>nine months ended</b></font></th></tr></table></body></html>